In [1]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import lxml
import os
from tqdm import tqdm
from selenium.webdriver.chrome.service import Service

import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date

In [2]:
# указать пользователя
user = 'MerinovDV'
path_to_credential = f'C:/Users/{user}/Downloads/auto-monitoring-367212-64ec4ad9d3a5.json' 

In [3]:
# Данные для доступа к Google Spreadsheets

# указать путь до credential
path_to_credential = path_to_credential 

# указать название таблицы, с которой будет работа
table_name = 'Сбор данных для презы hh2022'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)

gs = gspread.authorize(credentials)
work_table = gs.open(table_name)


In [4]:
def get_links(url, folder, user, add_folder=None, prof_obl=False):
    
    """
    функция проходит с помощью webdriver'a по всем ссылкам, считает кол-во страниц и на основании этого собирает
    все страницы с вакансиями для заданного запроса. После этого сохраняет страницы в папки
    
    url: str
        ссылка для запроса
    folder: str
        название папки
    user: str
        имя пользователя
    add_folder: str
        название дополнительной папки, если есть
    prof_obl: bool, default False
            True - если необходимо собирать данные из вкладки "Работа по профессиям"
    """
    
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0")
    options.add_argument("--disable-blink-features=AutomationControlled")

    s = Service(executable_path=f"C:/Users/{user}/Parsing_folder/chromedriver/chromedriver.exe")
    driver = webdriver.Chrome(service=s, options=options)
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        try:
            pages = int(soup.find_all(attrs={'class':'bloko-button','rel': 'nofollow', 'data-qa':'pager-page'})[-1].getText())
        except:
            pages = 1
        for page in range(pages):
            if prof_obl:
                driver.get(url+ '?page=' + str(page))
                time.sleep(0.25)
            else:
                driver.get(url+ '&page=' + str(page))
                time.sleep(0.25)
            
            if add_folder is not None:
                with open(f"C:/Users/{user}/hh/{add_folder}/{folder}/page_{page}_{date.today()}.html", 
                            "w", encoding='utf-8') as file:
                    file.write(driver.page_source)
            else:
                with open(f"C:/Users/{user}/hh/{folder}/page_{page}_{date.today()}.html", 
                            "w", encoding='utf-8') as file:
                    file.write(driver.page_source)
    except Exception as ex:
                print(ex)

    finally:
        driver.close()
        driver.quit()

In [5]:
def get_list_of_vacancy(folder, user, add_folder=None):
    
    """
    вовзращает список с ссылками на на вакансии
    
    folder: str
        название папки
    user: str
        имя пользователя
    add_folder: str
        название дополнительной папки, если есть
    """
    
    links_list = []
    if add_folder is None:
        path = f"C:/Users/{user}/hh/{folder}"
    else:
        path = f"C:/Users/{user}/hh/{add_folder}/{folder}"
        
    for html in os.listdir(path):
        with open(f"{path}/{html}", encoding='utf-8') as file:
            src = file.read()

            try:
                soup = BeautifulSoup(src, "lxml")
                result = soup.find_all("a", class_="serp-item__title")
                if len(result) != 0:
                    for i in range(len(result)):
                        link = result[i].get('href')
                        links_list.append(link)
                else:
                    print(f'[INFO_PARSER] нет данных по вакансиям {html}')

            except:
                print('[INFO] проблема с парсингом')  
    return links_list

In [6]:
def get_vacancy_info(dict_with_links_list):
    
    """
    функция возвращает DataFrame с описанием вакансии
    
    dict_with_links_list: dict
        словарь, в котором ключ - папка, а значение список ссылок на вакансии по заданному ключу
    """
    
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    
    dict_with_df = {}
    
    for key in dict_with_links_list:
        name = []
        salary_from = []
        salary_to = []
        salary_currency = []
        salary_gross = []
        experience_name = []
        schedule_name = []
        employment_name = []
        description = []
        key_skills = []
        employer = []
        published_at = []
        url = []
        views = []
        
        for link in tqdm(dict_with_links_list[key]):
            flag = False
            retry = 0
            while flag == False:
                r = requests.get(link, headers=headers).text
                soup = BeautifulSoup(r, 'lxml')
                if soup.find(attrs={'data-qa': 'vacancy-title'}) is not None:
                    
                    # название вакансии
                    title = soup.find(attrs={'data-qa': 'vacancy-title'})
                    if title is not None:
                        name.append(title.getText())
                    else:
                        name.append('null')

                    # зарплата
                    salary = soup.find(attrs={'data-qa': 'vacancy-salary'})
                    if salary is not None:
                        salary = salary.getText().replace(u'\xa0', u'').split(' ')
                        if 'от' in salary:
                            salary_from_i = salary.index('от') + 1
                            salary_from.append(salary[salary_from_i])
                        else:
                            salary_from.append('null')
                        if 'до' in salary:
                            salary_to_i = salary.index('до') + 1
                            if salary[salary_to_i] == 'вычета':
                                salary_to.append('null')
                            else:
                                salary_to.append(salary[salary_to_i])
                        else:
                            salary_to.append('null')
                    else:
                        salary_from.append('null')
                        salary_to.append('null')

                    # валюта
                    sal_currency = soup.find(attrs={'data-qa': 'vacancy-salary'})
                    if sal_currency is not None:
                        sal_currency = sal_currency.getText().split(' ')
                        if 'руб.' in sal_currency:
                            currency = 'RUB'
                        elif 'USD' in sal_currency:
                            currency = 'USD'
                        elif 'EUR' in sal_currency:
                            currency = 'EUR'
                        elif 'KZT' in sal_currency:
                            currency = 'KZT'
                        elif 'бел. руб' in sal_currency:
                            currency = 'BYN'
                        elif 'KGS' in sal_currency:
                            currency = 'KGS'
                        elif 'сум' in sal_currency:
                            currency = 'UZS'
                        elif 'AZN' in sal_currency:
                            currency = 'AZN'
                        else:
                            currency = 'null'
                        salary_currency.append(currency)
                    else:
                        salary_currency.append('null')

                    # тип зарплаты
                    salary_type = soup.find('span', attrs={'class': 'vacancy-salary-compensation-type'})
                    if salary_type is not None:
                        salary_gross.append(salary_type.getText().strip())
                    else: 
                            salary_gross.append('null')

                    # опыт работы
                    experience = soup.find(attrs={'data-qa': 'vacancy-experience'})
                    if experience is not None:
                        experience_name.append(experience.getText())
                    else: 
                        experience_name.append('null')

                    # тип занятости, график работы
                    employment_schedule = soup.find(attrs={'data-qa': 'vacancy-view-employment-mode'})
                    if employment_schedule is not None:
                        employment_name.append(employment_schedule.getText().split(', ')[0])
                        schedule_name.append(employment_schedule.getText().split(', ')[1])
                    else: 
                        employment_name.append('null')
                        schedule_name.append('null')

                    # описание вакансии
                    descrip = soup.find('div', class_='vacancy-section')
                    if descrip is not None:
                        description.append(descrip.getText())
                    else:
                        description.append('null')

                    # ключевые навыки
                    key_skills_i = []
                    skills = soup.find_all(attrs={'class': 'bloko-tag bloko-tag_inline'})
                    if len(skills) != 0:
                        for i in soup.find_all(attrs={'class': 'bloko-tag bloko-tag_inline'}):
                            key_skills_i.append(i.getText())
                        key_skills.append(key_skills_i)
                    else:
                        key_skills.append('null')

                    # название компании
                    emp = soup.find('span', class_='vacancy-company-name')
                    if emp is not None:
                        employer.append(emp.getText())
                    else:
                        employer.append('null')

                    # кол-во просмотров сейчас
                    view = soup.find('span', class_='vacancy-viewers-count')
                    if view is not None:
                        views.append(view.getText().replace(u'\xa0', u' ').split(' ')[0])
                    else:
                        views.append('null')

                    # дата публикации
                    dt = soup.find('p', class_='vacancy-creation-time-redesigned')
                    if dt is not None:
                        published_at.append(dt.getText().replace(u'\xa0', u' ')\
                                                    .split('опубликована')[1]\
                                                    .split('в')[0].strip())
                    else:
                        published_at.append('null')

                    url.append(link)

                    time.sleep(0.25)

                    
                    flag = True

                    
                retry += 1
                if retry == 5:
                    break
                    
        output = pd.DataFrame()
        output['name'] = name
        output['salary_from'] = salary_from
        output['salary_to'] = salary_to
        output['salary_currency'] = salary_currency
        output['salary_gross'] = salary_gross
        output['experience_name'] = experience_name
        output['schedule_name'] = schedule_name
        output['employment_name'] = employment_name
        output['description'] = description
        output['key_skills'] = key_skills
        output['employer'] = employer
        output['published_at'] = published_at
        output['url'] = url
        output['views'] = views
        output['type'] = key
        
        dict_with_df[key] = output

    return dict_with_df

In [7]:
def count_skills(column, to_df=True):
    
    """
    функция возвращает словарь: ключ - навык, значение - кол-во позиций в выгрузке
    
    column: str
        столбец в датафрейме, который содержит ключевые навыки
    to_df: bool, default True
        возвращает датафрейм, если True, иначе - словарь
    """
    
    dct = {}
    for skill_list in column:
        for skill in skill_list:
            if skill not in dct.keys():
                dct[skill] = 1
            elif skill in dct.keys():
                dct[skill] += 1
    if to_df:
        df_skills = pd.DataFrame.from_dict(dct, orient='index')\
            .rename(columns={0:'num_skills'})\
            .sort_values('num_skills', ascending=False)
        return df_skills
    else:
        return dct

In [8]:
def upload_df(df, google_sheet, create_sheet=False, skill_df=False):
    
    """
    функция загружает df в Google Spreadsheets
    
    df: pandas.DataFrame
        DataFrame, который необходимо загрузить
    google_sheet: str
        навзание листа в Google Spreadsheets
    create_sheet: bool, default False
        создает новый лист, если True, иначе загружает данные в текущий лист
        
    """
    if skill_df == False:
        df.key_skills = df.key_skills.astype('str')
        df.description = df.description.str.replace(u'\xa0', u' ').str.strip().str.replace(u'\n', u' ')
    else:
        pass
    
    if create_sheet:
        ws = work_table.add_worksheet(title=google_sheet, rows=100, cols=20)
    else:
        pass
    
    worksheet = work_table.worksheet(google_sheet)
    worksheet.append_row(df.columns.tolist(), value_input_option='USER_ENTERED')
    worksheet.append_rows(df.values.tolist(), value_input_option='USER_ENTERED')
    
    return print (f'Данные загружены на лист {google_sheet}')

# Пример выгрузки вакансий

In [10]:
# Получаем список ссылок
worksheet = work_table.worksheet('Ссылки')
val = worksheet.get_values('B222:C223')
columns = val.pop(0)
val = pd.DataFrame(val, columns=columns)

# создаем из них словарь
g_links_2 = dict(zip(val['название'].values, val['ссылки'].values))

# название папки для выгрузки
add_folder = 'программист с'

# создаем папки для страниц, если таких папок еще не было
for new_folder in val['название'].values:
    try:
        # создаем директорию
        os.mkdir(f'C:/Users/{user}/hh/{add_folder}/{new_folder}')
        print("Directory " , new_folder ,  " Created ") 
    except FileExistsError:
        print("Directory " , new_folder ,  " already exists")
        
# проходим по всем ключам словаря (папка - url) и вызываем функцию
for key in g_links_2:
    get_links(g_links_2[key], key, add_folder=add_folder, user=user)
    
# создаем новый словарь, в котором ключ - папка, а значение список ссылок на вакансии по заданному ключу
dict_with_links_list_g_2 = {}
for key in g_links_2:
    dict_with_links_list_g_2[key] = get_list_of_vacancy(key, add_folder=add_folder, user=user)
    
all_df = get_vacancy_info(dict_with_links_list_g_2)

Directory  Программист c  Created 


100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [01:20<00:00,  1.18s/it]


In [11]:
# объединяем полученные датафреймы
best_df = pd.DataFrame(columns=['name', 'salary_from', 'salary_to', 'salary_currency', 'salary_gross',
       'experience_name', 'schedule_name', 'employment_name', 'description',
       'key_skills', 'employer', 'published_at', 'url', 'views', 'type'])
for key in all_df:
    best_df = pd.concat([best_df, all_df[key]])

In [12]:
# считаем ключевые навыки
skills = pd.DataFrame()
for key in all_df:
    t = count_skills(all_df[key].key_skills).assign(type = key)
    skills = pd.concat([t, skills])
    

In [13]:
skills

,num_skills,type
Linux,46,Программист c
C/C++,34,Программист c
Git,25,Программист c
Английский язык,19,Программист c
C++,18,Программист c
...,...,...
Linux kernel,1,Программист c
FreeRTOS,1,Программист c
EXE,1,Программист c
WiFI,1,Программист c


In [14]:
upload_df(best_df, 'программист с', create_sheet=True)

Данные загружены на лист программист с


In [ ]:
upload_df(skills, 'программист с', create_sheet=True)